# Generate Balanced Parentheses

Given a non-negative integer n, return all well-formed parentheses strings that can be generated with
n pairs of parentheses.

**Example 1:**
```
n = 1
out = ["()"]
```

**Example 2:**
```
n = 2
out = ["(())", "()()"]
```

**Example 3:**
```
n = 3
out = ["((()))", "(()())", "(())()", "()(())", "()()()"]
```

In [1]:
from typing import List

## Recursion

- **Time Complexity:** $\mathrm{O}\left(4^n / \sqrt{n}\right)$
- **Space Complexity:** $\mathrm{O}(n)$ (for the recursion stack and building each string)

The number of valid parentheses strings for $n$ pairs is given by the $n$-th Catalan number:
$$
C(n) = \frac{1}{n+1} \binom{2n}{n} = \frac{(2n)!}{n! (n+1)!}
$$

The recursive function generates all valid combinations by exploring every possible placement of '(' and ')',
ensuring the parentheses remain balanced. The total number of unique combinations is $C(n)$.

Each valid string has length $2n$, so building each string takes $\mathrm{O}(n)$ time. However, the dominant
factor is the number of combinations, which grows rapidly with $n$.

Therefore, the overall time complexity is:
$$
\mathrm{O}(n \cdot C(n)) \approx \mathrm{O}\left(\frac{4^n}{\sqrt{n}}\right)
$$
where $$C(n) \approx \frac{4^n}{n^{3/2} \sqrt{\pi}}$$ for large $n$. The space complexity is $\mathrm{O}(n)$ due to the recursion depth and string construction.

In [4]:
def _gen_parens(
    accumulator: List[str], # Accumulates the well-formed parentheses
    builder: List[str],     # Holds the parentheses, used to build the well balanced string
    n_left: int,            # Number of left "(" remaining
    n_right: int,           # Number of right ")" remaining
    n_open: int,            # Number of unbalanced open parentheses "("
    cur: int,               # Current index of builder
):
    # The terminal condition
    if n_right == 0:
        accumulator.append("".join(builder))
        return
    
    # Build the left subtree
    if n_left > 0:
        builder[cur] = "("
        _gen_parens(
            accumulator=accumulator,
            builder=builder,
            n_left=(n_left - 1),
            n_right=n_right,
            n_open=(n_open + 1),
            cur=(cur + 1),
        )

    # Build the right subtree
    if n_open > 0 and n_right > 0:
        builder[cur] = ")"
        _gen_parens(
            accumulator=accumulator,
            builder=builder,
            n_left=n_left,
            n_right=(n_right - 1),
            n_open=(n_open - 1),
            cur=(cur + 1),
        )

In [8]:
def generate_parentheses(n: int) -> List[str]:
    accumulator = []
    builder = ["" for _ in range(2*n)]
    _gen_parens(
        accumulator=accumulator,
        builder=builder,
        n_left=n,
        n_right=n,
        n_open=0,
        cur=0,
    )
    return accumulator

In [9]:
generate_parentheses(0)

['']

In [ ]:
generate_parentheses(1)

['(())', '()()']

In [12]:
generate_parentheses(2)

['(())', '()()']

In [13]:
generate_parentheses(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

In [24]:
len(generate_parentheses(9))

4862

In [56]:
def gen_parens_recursive(n: int) -> List[str]:
    accumulator = []
    builder = ["" for _ in range(2 * n)]

    def _build(
        n_left: int,    # Number of left parentheses remaining
        n_right: int,   # Number of right parentheses remaining
        n_open: int,    # Number of unbalanced open parentheses
        cur: int,       # Current index being filled
    ):
        if n_right == 0:
            accumulator.append("".join(builder))
        
        # Build the left subtree
        if n_left > 0:
            builder[cur] = "("
            _build(n_left - 1, n_right, n_open + 1, cur + 1)

        # Build the right subtree
        if n_open > 0 and n_right > 0:
            builder[cur] = ")"
            _build(n_left, n_right - 1, n_open - 1, cur + 1)

    _build(n, n, 0, 0)
    return accumulator

In [57]:
gen_parens_recursive(0)

['']

In [58]:
gen_parens_recursive(1)

['()']

In [59]:
gen_parens_recursive(2)

['(())', '()()']

In [60]:
gen_parens_recursive(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

In [61]:
len(gen_parens_recursive(7))

429

In [62]:
len(gen_parens_recursive(8))

1430

## Brute Force

- **Time Complexity:** $\mathrm{O}(4^{n} \cdot n)$
- **Space Complexity:** $\mathrm{O}(n)$ (for recursion stack and string building)

The brute force approach generates all possible sequences of $2n$ parentheses, regardless of validity. For each sequence, it checks if the parentheses are balanced, which takes $\mathrm{O}(n)$ time per sequence. Since there are $2^{2n}$ possible sequences, the total time complexity is $\mathrm{O}(2^{2n} \cdot n)$. The space complexity is $\mathrm{O}(2n)$ due to the recursion depth and the temporary string used to build each sequence. This method is much less efficient than the recursive approach that only generates valid combinations.

In [49]:
def gen_parens_brute_force(n: int) -> List[str]:
    accumulator = []
    b = ["" for _ in range(2 * n)]

    def valid(k: int):
        n_open = 0
        for i in range(k):
            n_open += (1 if b[i] == "(" else -1)
            if n_open < 0:
                return False
        return n_open == 0
    
    def backtrack(i: int):
        if i == 2*n:
            if valid(2 * n):
                accumulator.append("".join(b))
            return
        b[i] = "("
        backtrack(i + 1)
        b[i] = ")"
        backtrack(i + 1)

    backtrack(0)
    return accumulator


In [50]:
gen_parens_brute_force(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

In [48]:
len(gen_parens_brute_force(8))

1430

In [68]:
%%timeit
len(generate_parentheses(10))

17.4 ms ± 523 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%%timeit
len(gen_parens_recursive(10))

14.7 ms ± 778 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%%timeit
len(gen_parens_brute_force(10))

649 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
